<a href="https://colab.research.google.com/github/SushilKokare/AtiNeu_Project/blob/main/8_Vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten
from keras.models import Sequential
import os
from keras import backend as k
from keras.applications.vgg16 import VGG16

In [39]:
pathTrain = '/content/drive/MyDrive/AtiNeu Project/Data/MRI/Brain_Data_Organised/Train'
pathTest = '/content/drive/MyDrive/AtiNeu Project/Data/MRI/Brain_Data_Organised/Test'

In [40]:
img_size = (224,224)
batch_size = 32

In [41]:
if k.image_data_format() == 'channels_first':
  input_shape = (3,img_size[0],img_size[1])
else:
  input_shape = (img_size[0],img_size[1],3)

In [42]:
train_gen = ImageDataGenerator(rescale = 1./255)
test_gen = ImageDataGenerator(rescale = 1./255)

In [54]:
train_data = train_gen.flow_from_directory(
    pathTrain,
    target_size = img_size,
    batch_size = batch_size,
    # color_mode = 'grayscale',
    class_mode = 'binary',
    classes = ['Normal','Stroke']
)

Found 2251 images belonging to 2 classes.


In [55]:
test_data =test_gen.flow_from_directory(
    pathTest,
    target_size = img_size,
    # color_mode = 'grayscale',
    class_mode = 'binary',
    classes = ['Normal','Stroke'],
    batch_size = batch_size
)

Found 250 images belonging to 2 classes.


In [56]:
base_model = VGG16(weights = 'imagenet',include_top = False , 
                   input_shape=input_shape)

In [57]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [58]:
for layer in base_model.layers:
  layer.trainable = False

In [59]:
model.compile(optimizer = 'adam' , loss='binary_crossentropy',
              metrics=['accuracy'])

In [60]:
import numpy as np

In [61]:
train_steps_per_epoch  = np.ceil(train_data.samples / batch_size)
test_steps_per_epoch = np.ceil(test_data.samples / batch_size)

In [62]:
train_data.samples

2251

In [63]:
test_data.n

250

In [65]:
history = model.fit(train_data,epochs=10,validation_data = test_data,
          steps_per_epoch=train_steps_per_epoch,
          validation_steps = test_steps_per_epoch)

Epoch 1/10
71/71 [==============================] - 14s 193ms/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 0.6119 - val_accuracy: 0.8400
Epoch 2/10
71/71 [==============================] - 12s 171ms/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.5757 - val_accuracy: 0.8400
Epoch 3/10
71/71 [==============================] - 12s 171ms/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 0.7237 - val_accuracy: 0.8120
Epoch 4/10
71/71 [==============================] - 12s 170ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.7715 - val_accuracy: 0.8040
Epoch 5/10
71/71 [==============================] - 13s 185ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.7263 - val_accuracy: 0.8200
Epoch 6/10
71/71 [==============================] - 12s 170ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.6508 - val_accuracy: 0.8280
Epoch 7/10
71/71 [==============================] - 12s 165ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.7702 - val_accuracy: 0.8200

In [68]:
# Evaluate the model on the test set
scores = model.evaluate(test_data, steps=test_steps_per_epoch)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


8/8 [==============================] - 2s 163ms/step - loss: 0.7349 - accuracy: 0.8240
Test loss: 0.7348811626434326
Test accuracy: 0.8240000009536743


In [69]:
import matplotlib.pyplot as plt

In [75]:
# plt.figure()
# plt.xlabel('epoch')
# plt.ylabel('values')
# for i in ['accuracy','val_accuracy']:
#   x = history.history[i]
#   plt.plot(x,label=i)
# plt.legend()